# Using Arbitrum with Agents

In the previous tutorial, we have seen how to use the `GizaAgent` class to create a simple agent that can interact with the Ethereum blockchain. In this tutorial, we will learn how to use other chains with the `GizaAgent` class.

As we rely on `ape` to interact with the blockchain, we can use any chain that it supports. The list of supported chains via plugins can be found [here](https://docs.apeworx.io/).

In this tutorial, we will use `Arbitrum` as an example. `Arbitrum` is a layer 2 solution for Ethereum that provides low cost and fast transactions. It is supported by `ape` and we can use it with the `GizaAgent` class.

## Before you begin

1. Python 3.11 or later must be installed on your machine
2. Giza CLI must be installed on your machine. You can install it by running `pip install giza-cli`
3. giza-agents should be installed with the extra `agents`. You can install it by running `pip install giza-agents[ape]`
4. You must have an active Giza account. If you don't have one, you can create one [here](https://cli.gizatech.xyz/examples/basic).
5. You must have a model deployed on Giza. You can follow the tutorial [Build a Verifiable Neural Network with Giza Actions](https://actions.gizatech.xyz/tutorials/build-a-verifiable-neural-network-with-giza-agents) to deploy an MNIST model on Giza.
6. During the tutorial, you will need to interact with the Giza CLI, Ape's framework, and provide multiple inputs, like `model-id`, `version-id`, `account name`, etc.
7. You must be logged in to the Giza CLI or have an API KEY.

## Installing the required libraries

Let's start by installing the required libraries.

```bash
pip install giza-agents[ape]
```

This will install the `giza-agents` library along with the `agents` extra, which contains the necessary tools to create an AI Agent.

## Creating an account

Before we can create an AI Agent, we need to create an account using Ape's framework. We can do this by running the following command:

```bash
$ ape accounts generate <account name>
Enhance the security of your account by adding additional random input:
Show mnemonic? [Y/n]: n
Create Passphrase to encrypt account:
Repeat for confirmation:
SUCCESS: A new account '0x766867bB2E3E1A6E6245F4930b47E9aF54cEba0C' with HDPath m/44'/60'/0'/0/0 has been added with the id '<account name>'
```

This will create a new account under `$HOME/.ape/accounts` using the keyfile structure from the [eth-keyfile library](https://github.com/ethereum/eth-keyfile)
. For more information on the account management, you can refer to the [Ape's framework documentation](https://docs.apeworx.io/ape/stable/userguides/accounts.html#keyfile-accounts).

During account generation we will be prompted to enter a passphrase to encrypt the account, which will be used to unlock the said account when needed, so **make sure to keep it safe**.

We encourage the creation of a new account for each agent, as it will allow you to manage the agent's permissions and access control more effectively, but importing accounts is also possible.

## Creating an AI Agent

Now that we have a funded account, we can create an AI Agent. This can be done by running the following command:

```bash
giza agents create --model-id <model-id> --version-id <version-id> --name <agent name> --description <agent description>

# or if you have the endpoint-id

giza agents create --endpoint-id <endpoint-id> --name <agent name> --description <agent description>
```

This command will prompt you to choose the account you want to use with the agent. Once you select the account, the agent will be created and you will receive the agent id. The output will look like this:

```console
[giza][2024-04-10 11:50:24.005] Creating agent ✅
[giza][2024-04-10 11:50:24.006] Using endpoint id to create agent, retrieving model id and version id
[giza][2024-04-10 11:50:53.480] Select an existing account to create the agent.
[giza][2024-04-10 11:50:53.480] Available accounts are:
┏━━━━━━━━━━━━━┓
┃  Accounts   ┃
┡━━━━━━━━━━━━━┩
│ my_account  │
└─────────────┘
Enter the account name: my_account
{
  "id": 1,
  "name": <agent_name>,
  "description": <agent_description>,
  "parameters": {
    "model_id": <model_id>,
    "version_id": <version_id>,
    "endpoint_id": <endpoint_id>,
    "alias": "my_account"
  },
  "created_date": "2024-04-10T09:51:04.226448",
  "last_update": "2024-04-10T09:51:04.226448"
}
```

This will create an AI Agent that can be used to interact with the deployed MNIST model.

## Use Agents in Arbitrum

Let's start by installing the `ape-arbitrum` plugin.

In [ ]:
!ape plugins install arbitrum

We can confirm whether it has been installed, by running `ape networks list` in the terminal.

In [2]:
!ape networks list

arbitrum
├── goerli
│   ├── alchemy
│   └── geth  (default)
├── local  (default)
│   └── test  (default)
├── mainnet
│   ├── alchemy
│   └── geth  (default)
└── sepolia
    ├── alchemy
    └── geth  (default)
ethereum  (default)
├── goerli
│   ├── alchemy
│   └── geth  (default)
├── local  (default)
│   ├── geth
│   └── test  (default)
├── mainnet
│   ├── alchemy
│   └── geth  (default)
└── sepolia
    ├── alchemy
    └── geth  (default)


Here, we can see that we have multiple networks available, including `arbitrum`. So now we can use it when instantiating the `GizaAgent` class.

For this execution, we will use the Arbitrum mainnet and a private RPC node. This is because public nodes have small quotas that can easily be reached.

The contract is a verified contract selected at random from [arbiscan](https://arbiscan.io/). The aim is to show that we can read properties from this contract, which means that we could also execute a write function.

In this case, as we are only executing a read function, there is no need for a funded wallet because we won't be signing any transactions.

Remember that you will need to specify the `<Account>_PASSPHRASE`. If you wish to launch your operation as a script, exporting it will be enough:

```bash
export <Account>_PASSPHRASE=your-passphrase
```

If you are using it from a notebook, you will need to launch the notebook instance from an environment with the passphrase variable or set it in the code before importing `giza.agents`:

```python
import os
os.environ["<Account>_PASSPHRASE"] = "your-passphrase"

from giza.agents import GizaAgent
...
```

Now we can instantiate the agent:

In [5]:
import os
os.environ["<Account>_PASSPHRASE"] = ...


from giza.agents import GizaAgent

MODEL_ID = ...
VERSION_ID = ...
ACCOUNT = ...
PRIVATE_RPC = ... # This can also be loaded from the environment or a .env file

agent = GizaAgent(
    id=MODEL_ID,
    version_id=VERSION_ID,
    chain=f"arbitrum:mainnet:{PRIVATE_RPC}",
    account=ACCOUNT,
    contracts={
        "random": "0x8606d62fD47473Fad2db53Ce7b2B820FdEab7AAF"
    }
)

Now that we have the agent instance, we can enter the `execute()` context and call the read function from an Arbitrum smart contract:

In [6]:
with agent.execute() as contracts:
    result = contracts.random.name()
    print(f"Contract name is: {result}")

INFO: Connecting to a 'nitro' node.
Contract name is: FatalDoge


## What we have learned

In this tutorial, we learned how to create an AI Agent and interact with Arbitrum.

For this, we needed to:

* Install the `arbitrum` plugin
* Check that the new network is available
* Got a contract from `arbiscan`
* Use an agent to execute a function from an `arbitrum` smart contract

Now these same steps can be followed to use any other network supported by `ape` and interact with different chains.